In [1]:
!sudo pip install transformers --upgrade

     |████████████████████████████████| 7.9MB 6.5MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 106.5MB/s eta 0:00:01
  Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [2]:
#import torch
#filename = '/data/artifacts_gpu/training_artifacts_T5_news1.pkl'
#model = torch.load(filename,map_location=torch.device('cpu'))

In [3]:
import torch
from transformers import pipeline

# load model
model = AutoModelForCausalLM.from_pretrained(
    '/data/mistral/query-to-mql/exp-8/oct-30/merged-model',
#     quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
pipe("Reason for sales drop  Quarter1")

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2574: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /hom

[{'generated_text': 'Reason for sales drop  Quarter1: Given the context of the problem, we can assume that'}]

In [4]:
import torch
from transformers import pipeline



#pipe = pipeline("text-generation", model="/data/mistral/query-to-mql/exp-8/oct-30/merged-model", torch_dtype=torch.bfloat16, device_map="auto")
#pipe = pipeline("text-generation", model="/data/mistral/query-to-mql/exp-8/oct-30/merged-model", torch_dtype=torch.float16, device_map="cpu")
#pipe = pipeline("text-generation", model="/data/mistral/query-to-mql/exp-8/oct-30/merged-model",device_map="cpu")
pipe = pipeline("text-generation", model="/data/mistral/query-to-mql/exp-8/oct-30/merged-model", torch_dtype="auto", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
context_data = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

In [6]:
date_input_data = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [7]:
query_template_v1 = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [8]:
model_name="/data/mistral/query-to-mql/exp-8/oct-30/merged-model"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
def predict_template_query_v1(user_query):
    prompt = query_template_v1.format(context=context_data,
                                   user_query=user_query,
                                  date_input=date_input_data)
    _inputs = tokenizer.encode(prompt, return_tensors="pt")
#   outputs = model.generate(input_ids=_inputs.to('cuda'), max_length= 1700, pad_token_id=tokenizer.eos_token_id)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
    output = tokenizer.decode(outputs[0])
    output_new = output.split('[MQL]\n')[1]
    print(output_new.split('\n[/MQL]')[0])
    return output_new.split('\n[/MQL]')[0], output
    
#     return output

In [12]:
%%time
user_query = 'why sales changed in last 2 weeks of aug 2021'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

Setting `pad_token_id` to `eos_token_id`:49155 for open-end generation.


user query:  why sales changed in last 2 weeks of aug 2021
----------------------------------------------------------------------------------------------------


ValueError: invalid literal for int() with base 10: 'generated_text'

In [11]:
# We use a variant of ChatML to format each message
import time

t1 = time.time()

prompt_template = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
{mql}
[/MQL]
the steps and rationale used to achieve above structured output is as below.
{rationale}
"""

prompt = prompt_template.format(context=context_data,date_input=date_input_data,user_query = 'what will be sales of phone in 10.21')
print("before model start")
# We use a special <|end|> token with ID 49155 to denote ends of a turn
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)

t2 = time.time()
t2-t1

KeyError: 'mql'

In [ ]:
outputs[0]["generated_text"].split("\n")

In [6]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
class ScoreTemplateExample(ScoreBase):
#     """
#     This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
#     """    
     def __init__(self):
        super().__init__()
        
        self.model_loaded = None
        
        import torch
        from transformers import pipeline
        
        # load model
        model = AutoModelForCausalLM.from_pretrained(
            '/data/mistral/query-to-mql/exp-8/oct-30/merged-model',
        #     quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        model.config.use_cache = False
        model.config.pretraining_tp = 1

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", trust_remote_code=True,
                                                  # add_eos_token=True,
                                                  use_fast=False)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"

        # pipeline
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map='auto')

 #############################################################       
 #       if self.model_loaded is None:
 #           print("LLM model loading from data section")
 #           self.model_loaded = pipeline("text-generation", model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f", torch_dtype=torch.bfloat16, device_map="auto")
#######################################################################
     def request_processing_fn(self, request) :
#         """
#         Processes Request Object -> List[Input data, ..]. It could be:
#                A List Mapping of All Value Can Be one of : 
#                    - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
#                    - List[numpy.array(), numpy.array(), ...]
#                    - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
#                    - List[ SingleSample, SingleSample]
                   
#         :return: (n_inputs, payload's)
        
#         Warnings:
#         1. Do not reshape your final output for single sample here, do it in prediction.
#            Else payloads will be invalidated for extraction at raw and extraction level.
#         """
         final_payload = []
         raw_payload = request.json["payload"]
         return (1, raw_payload) 
    
     def pre_processing_fn(self,payload):
#         # All preprocessing step must occur in this section
#         # Takes Single Sample -> Returns Single Sample
        
#         # Not Doing Any Preprocessing Hence Returned payload
#         print("payload is ", payload)
        
         return payload

     def prediction_fn(self,
                       model: Any,
                       input_query 
                       ):
#         """
#                 Does the main prediction on pre_processed_input(Single Sample) using supplied model .

#                 :param model: Supported Model
#                 :param pre_processed_input: Single Preprocessed Payload
#                 :return: Prediction Value From the model
                
#                 Important Notes:
#                 - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
#         """
#         model_loaded = self.model_loaded
#         mod = model_loaded
#         text = pre_processed_input #this is tuple we can iterate if there is number of input
        
    
##################################################################################################    
#          prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
#          text = input_query
#          prompt = prompt_template.format(query=text)
#         # We use a special <|end|> token with ID 49155 to denote ends of a turn
#          outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
#          preds = str(outputs[0]["generated_text"].split("<|assistant|>")[1])
        
        
#          print("prediction is \n: ",preds)
    
#          return preds
######################################################################################################

#     class Meta:    
         # List of Callables() can be attached For Calling After AnSd Before Scoring
#         def __init__(self):
#             self.name = "Pre Hooked Me !"
#             self.pre_call_hooks.append(self.print_)
#         def print_(self):
#             print(self.name)
#             pre_call_hooks = []
#             post_call_hooks = []
######################################################################################################
        # pipeline
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map='auto')
        output=pipe('Promopt')
        return output

In [ ]:
score_ = ScoreTemplateExample()
import requests
req = requests.Request()

req.json = {"payload":"How do I sort a list in Python?"}